# crop damage prediction 
# Alive = 0  damage due to other reason =1 damage due to pesticide=2 

## Importing dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Loading trainingset and test set

In [ ]:

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

# EDA on training set

In [ ]:
# checking the shape of the training
train.shape

In [ ]:
# print 1st 6 lines of traoning set
train.head(n=6)

In [ ]:
# get info about variors attribute and respctive dtype
train.info()

In [ ]:
# basis statistical analysis on training set
train.describe()

In [ ]:
# checking if there is any null values in training set
train.isnull().sum()

In [ ]:
#heat map repesntstion of null values
sns.heatmap(train.isnull(),yticklabels=False)

In [ ]:
# to get number of unique values in training set attributes
train.iloc[:,0:-1].nunique()


In [ ]:
# to get number of unique values traget variable
train.iloc[:,-1].unique()

In [ ]:
# Histogram of 'Estimated_Insects_Count'
sns.set_style('darkgrid')
sns.distplot(train['Estimated_Insects_Count'],kde=False)

In [ ]:
# Histogram of 'Crop_Type'
sns.set_style('darkgrid')
sns.distplot(train['Crop_Type'],kde=False)

In [ ]:
# Histogram of 'Soil_Type'
sns.set_style('darkgrid')
sns.distplot(train['Soil_Type'],kde=False)

In [ ]:
# Histogram of 'Number_Doses_Week of pesticides'
sns.distplot(train['Number_Doses_Week'],kde=False)
# 20 doses per week is usesd mostly

In [ ]:
# Histogram of 'Crop_Damage'
sns.distplot(train['Crop_Damage'],kde=False)
# most of the crop are alive ~0
# little more than 10000 is damaged due to other reasons ~ 1
# very few number of crop are damaged due to pesticide ~ 2

# our target should to develop  a model so that it can generalise the prediction
# predict all 3 classes especially class "2" which have relativly few training eg 

In [ ]:
# Box plot of 'Crop_Damage' vs 'Number_Weeks_Used'
sns.set_style('darkgrid')
sns.boxplot(x='Crop_Damage',y='Number_Weeks_Used',data=train )


In [ ]:
# Box plot of 'pesticides use catagory' vs 'Number_Weeks_Used'
# 1~never used
# 2~previously used
# 3~ currently in use
sns.set_style('darkgrid')
sns.boxplot(x='Pesticide_Use_Category',y='Number_Weeks_Used',data=train )
#  median class 1 ~ 0 
#  median class 2 ~ 25 
#  median class 3 ~ 35
# these value r crucial for data cleaing


In [ ]:
# correlation coefficent 
train['Number_Weeks_Used'].corr(train['Pesticide_Use_Category'])

In [ ]:
# to check how pesticide affect crop damage
sns.countplot(x=train['Crop_Damage'],hue=train['Pesticide_Use_Category'])

In [ ]:
# to check how season affect crop damage
sns.countplot(x=train['Crop_Damage'],hue=train['Season'])

## Data preprocessing

In [ ]:
# "ID" value is not required for prediction
train.drop('ID',axis=1,inplace=True)

### Filling Missing values

In [ ]:
# when pesticide class==1 then  Number_Weeks_Used=0
# when pesticide class==2 then  Number_Weeks_Used=25
# when pesticide class==3 then  Number_Weeks_Used=35
def missing(c):
    Number_Weeks_Used=c[0]
    Pesticide_Use_Category=c[1]
    if pd.isnull(Number_Weeks_Used):
        if Pesticide_Use_Category==1:
            return 0
        elif Pesticide_Use_Category==2:
            return 25
        else:
            return 35
    else:
        return Number_Weeks_Used

In [ ]:
# applying missing( ) to traing set and test set 
train['Number_Weeks_Used']=train[['Number_Weeks_Used','Pesticide_Use_Category']].apply(missing,axis=1)
test['Number_Weeks_Used']=test[['Number_Weeks_Used','Pesticide_Use_Category']].apply(missing,axis=1)

In [ ]:
# to check if all null values are filled
train.isnull().sum()

In [ ]:
# separating  attribute(x) and labels(y)
x=train.iloc[:,:-1].values
y=train.iloc[:,-1].values
x_test=test.iloc[:,1:].values

In [ ]:
# spliting x and y into x_train , y_train and cross validation set (x_val,y_val)
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=100)

In [ ]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_val=sc.transform(x_val)
x_test=sc.transform(x_test)

In [ ]:
# import random forest clasifier 
from sklearn.ensemble import RandomForestClassifier
cal2=RandomForestClassifier(n_estimators=100,random_state=42,criterion='entropy') # creating object "cal2" of the class
cal2.fit(x_train,y_train) # fitting x_training & y_train to "cal2"

In [ ]:
# import adaptive boosting to improve model
from sklearn.ensemble import AdaBoostClassifier
boost=AdaBoostClassifier(base_estimator=cal2,n_estimators=2, learning_rate=1.0, algorithm='SAMME.R', random_state=None)
boost.fit(x_train,y_train)

In [ ]:
# predicting the values of x_val
y_pred2=boost.predict(x_val)

In [ ]:
# getting confussion matrix and accuracy score
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_val,y_pred2)
print(cm)
accuracy_score(y_val,y_pred2)

In [ ]:
# procduing classification report of model
from sklearn.metrics import classification_report
print(classification_report(y_val,y_pred2))

In [ ]:
# precicting test set
pred=boost.predict(x_test)

In [ ]:
#writing prediction into "sample_submission.csv"
df=pd.DataFrame(pred,test['ID'])
df.rename(columns={0:'Crop_Damage'},inplace=True)
df.to_csv('sample_submission.csv')

In [ ]:
print(df)